In [4]:
import time
import traceback
import datetime
import copy
import sys
import pickle
import os
import yaml
import pandas as pd
import numpy as np
import lightgbm as lgb
from pprint import pprint

import ccxt

In [89]:
# 必要な情報を読み込み
def fetch_yaml_config() -> dict:
    """read yaml config file"""
    config = "config.yml"
    with open(config, "r", encoding="utf-8") as yml:
        config = yaml.safe_load(yml)
    return config

In [99]:
def get_position(exchange: str, ex: ccxt) -> dict:
        """
        Return
        ---------------------------
        dict or None : 
            dict[float, float]
                qty: float 
                    position size 
                avgEntry: float
                    average entry price
        """
        if exchange == "bitflyer":
            position_response = ex.private_get_getpositions(params={"product_code" : "FX_BTC_JPY"})
            # [
            #     {
            #         "product_code" : "FX_BTC_JPY", 
            #         ''side" : "BUY", 
            #         ''price" : 36000, 
            #         ''size" : 10, 
            #         ''commission" : 0, 
            #          "swap_point_accumulate" : -35, 
            #          "require_collateral" : 120000, 
            #          "open_date" : "2015-11-03T10:04:45.011", 
            #          "leverage" : 3, 
            #          "pnl" : 965, 
            #          "sfd" : -0.5
            #     }
            # ]

            if not position_response:
                # 現在ポジションを持っていない状態
                net_size = 0.0
                avg_entry = 0.0
            else:
                # 現在ポジションを持っている状態
                size = []
                price = []
                for pos in position_response:
                    size.append(pos['size'])
                    price.append(pos['price'])
                    side = pos['side']
                
                # 平均建値を計算する
                avg_entry = round(
                    sum(
                        price[i] * size[i] for i in range(len(price))
                    ) / sum(size)
                )
                # 建玉の計算
                net_size = round(sum(size),2)
                if side == 'SELL':
                    net_size = net_size * -1.0
            
            return {'qty' : net_size, 'avgEntry' : avg_entry}

In [98]:
def exec_order(order_direction: str, ex: ccxt, exchange: str, order_lot: float, max_position_size: float, symbol: str):
    try:
        position = get_position(exchange, ex)
        position_size = position['qty']

        # 1. Longの注文 2. ショートポジションを持っていない 3. 現在のポジションに買い注文を足したものがMAXポジションサイズを超えないこと
        if order_direction == 'buy' and position_size >= 0.0 and position_size + order_lot <= max_position_size:
            order = ex.create_order(
                symbol=symbol,
                type='market',
                side=order_direction,
                amount=order_lot,
                params = { "product_code" : "FX_BTC_JPY" }
            )
        # 1. Shortの注文 2. ロングポジションを持っていない 3. 現在のポジションに売り注文を足したものがMAXポジションサイズを超えないこと
        if order_direction == 'sell' and position_size <= 0.0 and position_size - order_lot <= max_position_size * -1.0:
            order = ex.create_order(
                symbol=symbol,
                type='market',
                side=order_direction,
                amount=order_lot,
                params = { "product_code" : "FX_BTC_JPY" }
            )
    except KeyboardInterrupt:
        sys.exit()
    except Exception as e:
        traceback.print_exc() 

In [73]:
def check_sfd(ex: ccxt, symbol: str) -> float:
    fx_close = ex.fetch_ticker(symbol=symbol, params = { "product_code" : "FX_BTC_JPY" })['close']
    close = ex.fetch_ticker(symbol=symbol, params = { "product_code" : "BTC_JPY" })['close']
    sfd = (fx_close - close) / fx_close
    return sfd

In [88]:
if __name__ == "__main__":
    config = fetch_yaml_config()
    # 環境変数を設定する
    api_key = config["API_KEY"]
    api_secret = config["API_SECRET"]
    order_lot = float(config["LOT"]) # 0.001
    symbol = config["SYMBOL"] # BTC/JPY
    exchange = config["EXCHANGE"] # bitflyer
    long_th = config["LONG_THRESHOLD"] # -0.048
    short_th = config["SHORT_THRESHOLD"] # 0.048
    max_position_size = config["MAX_POSITION_SIZE"] # 0.005

    # bitFlyerのccxtクラスオブジェクトを生成
    ccxt_config = {"apiKey": api_key, "secret": api_secret}
    ex = ccxt.bitflyer(ccxt_config)

    periods_dict = {"1m" :60, "3m":60*3, "5m":60*5, "15m":60*15, "30m":60*30, "1h":60*60, "2h":60*60*2, "4h":60*60*4, "1d":60*60*24}

    period = "1m"
    timespan = periods_dict[period]
    t = time.time()
    pre_time = t//timespan
    
    while True:
        t = time.time()
        now_time = t//timespan
        if pre_time != now_time:
            pre_time = now_time
            sfd = check_sfd(ex, symbol)
            # PositionCondition
            if long_th > sfd:
                # 買い注文
                exec_order('buy', ex, exchange, order_lot, max_position_size, symbol)
            elif short_th < sfd:
                # 売り注文
                exec_order('sell', ex, exchange, order_lot, max_position_size, symbol)
            
        time.sleep(1)

<class 'dict'>


KeyboardInterrupt: 

In [90]:
res = ex.private_get_getpositions( params = { "product_code" : "FX_BTC_JPY" })
print(res)

[]


In [86]:
collateral = ex.private_get_getcollateral()
pprint( collateral )

{'collateral': '0.0',
 'keep_rate': '0.0',
 'margin_call_amount': '0.0',
 'margin_call_due_date': None,
 'open_position_pnl': '0.0',
 'require_collateral': '0.0'}


In [8]:
today_now = datetime.datetime.now()
today_now

datetime.datetime(2022, 7, 3, 10, 33, 6, 611652)